In [ ]:
#1. 사용환경 준비
import os
from dotenv import load_dotenv
from openai import OpenAI 
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import LLMChain


# openai API키 입력
dotenv_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..', '.env')
load_dotenv(dotenv_path)

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [ ]:

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:


# PDF 파일 로드. 파일의 경로 입력
loader = CSVLoader("../크롤링_자료/문화재_크롤링_결과.csv",encoding='UTF8')
loader2 = CSVLoader("../크롤링_자료/여행지_크롤링_결과",encoding='UTF8')

# 페이지 별 문서 로드
Treasures = loader.load()
Travel = loader2.load()
print(Travel[:1])

[Document(metadata={'source': 'place.csv', 'row': 0}, page_content='title: 구인사(단양)\naddress: 충청북도 단양군 영춘면 구인사길 73\ntelNo: ')]


In [ ]:


# 텍스트 청크 분할기 설정 (문단 기준 분할)
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(Treasures) # 문서를 청크로 분할
splits2 = text_splitter.split_documents(Travel)
print(splits2[1]) # 상위 10개만 출력

page_content='title: 원대리 자작나무 숲 (속삭이는 자작나무 숲)
address: 강원특별자치도 인제군 인제읍 자작나무숲길 760
telNo: 자작나무숲 안내소 033-463-0044' metadata={'source': 'place.csv', 'row': 1}


In [ ]:

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [ ]:

# 문서에서 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)


In [30]:
#7. FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [ ]:
#9. LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성하라. 


class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_Treasures = self.retriever.get_relevant_documents(query)
        return response_Treasures
#텍스트 생성 체인 생성
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

In [33]:
#10. 질문에 응답하는 챗봇을 구동하여 질문해라. 

# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    if query == "종료": # 종료 입력 시 챗봇 종료
        break
    
    # 1. Retriever로 관련 문서 검색
    response_Treasures = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_Treasures,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n질문:",query)
    print("\n답변:")
    print(response.content) # 답변 출력


질문: 숭례문 주변 여행지가 어디 있을까?

답변:
숭례문 주변 여행지로는 경복궁, 덕수궁, 남산서울타워, 명동 등이 있습니다.

질문: 부산 관련 국보나 보물은 뭐가 있을까?

답변:
부산 관련 국보나 보물로는 '조선왕조실록 태백산사고본'이 있습니다. 이 문서는 부산 연제구의 국가기록원 역사기록관에 위치해 있습니다.

질문: 그럼 거기 관련 여행지는 뭐가 있을까?

답변:
해당 지역인 경기 성남시 분당구 하오개로에 위치한 한국학중앙연구원 주변의 관련 여행지는 다음과 같습니다:

1. 한국학중앙연구원 - 한국의 역사와 문화에 대한 연구와 자료를 제공하는 기관으로, 다양한 전시와 행사도 열립니다.
2. 성남시청 - 지역의 행정과 문화 행사 관련 정보를 접할 수 있는 곳입니다.
3. 분당 중앙공원 - 자연을 즐기고 산책할 수 있는 공원으로, 여유로운 시간을 보낼 수 있습니다.
4. 판교 스타필드 - 쇼핑과 다양한 레저 시설이 있는 복합 공간으로, 가족 단위 방문객에게 적합합니다.

이 외에도 성남시는 다양한 문화재와 공원, 카페 등이 있어 여행지로 탐방하기에 좋은 곳입니다.

질문: 조선왕조실록 태백산사고본 근처 여행지를 알려줘

답변:
조선왕조실록 태백산사고본은 부산 연제구 경기장로 28에 위치한 국가기록원 역사기록관에 있습니다. 이 지역 근처의 여행지로는 부산의 유명한 관광지인 광안리 해수욕장, 해운대 해수욕장, 그리고 부산타워가 있습니다. 또한, 근처에 있는 연산동의 동래 온천과 같은 명소도 방문할 수 있습니다.
